In this project I'm learning how to create a reproducable ML model pipeline using MLflow to track experiments including logging hyperparameters and model versions. This allowed me to explore the effect of different feature engineering techniques and ways to deal with class imbalance in the dataset

In [1]:
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, make_scorer, recall_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import ADASYN, SMOTE
import warnings
warnings.filterwarnings("ignore")

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [3]:
df = pd.read_csv("FraudDetectionTransactionsDataset.csv")
df.head()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,...,7,437.63,3,Amex,65,883.17,Biometric,0.8494,0,0
1,TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-07 04:01:00,75725.25,Mobile,New York,Clothing,0,...,13,478.76,4,Mastercard,186,2203.36,Password,0.0959,0,1
2,TXN_199,USER_2734,28.96,Online,2023-06-20 15:25:00,1588.96,Tablet,Mumbai,Restaurants,0,...,14,50.01,4,Visa,226,1909.29,Biometric,0.8400,0,1
3,TXN_12447,USER_2617,254.32,ATM Withdrawal,2023-12-07 00:31:00,76807.20,Tablet,New York,Clothing,0,...,8,182.48,4,Visa,76,1311.86,OTP,0.7935,0,1
4,TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44:00,92354.66,Mobile,Mumbai,Electronics,0,...,14,328.69,4,Mastercard,140,966.98,Password,0.3819,1,1


In [4]:
df.columns

Index(['Transaction_ID', 'User_ID', 'Transaction_Amount', 'Transaction_Type',
       'Timestamp', 'Account_Balance', 'Device_Type', 'Location',
       'Merchant_Category', 'IP_Address_Flag', 'Previous_Fraudulent_Activity',
       'Daily_Transaction_Count', 'Avg_Transaction_Amount_7d',
       'Failed_Transaction_Count_7d', 'Card_Type', 'Card_Age',
       'Transaction_Distance', 'Authentication_Method', 'Risk_Score',
       'Is_Weekend', 'Fraud_Label'],
      dtype='object')

In [5]:
df.columns = [x.lower() for x in df.columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   transaction_id                50000 non-null  object 
 1   user_id                       50000 non-null  object 
 2   transaction_amount            50000 non-null  float64
 3   transaction_type              50000 non-null  object 
 4   timestamp                     50000 non-null  object 
 5   account_balance               50000 non-null  float64
 6   device_type                   50000 non-null  object 
 7   location                      50000 non-null  object 
 8   merchant_category             50000 non-null  object 
 9   ip_address_flag               50000 non-null  int64  
 10  previous_fraudulent_activity  50000 non-null  int64  
 11  daily_transaction_count       50000 non-null  int64  
 12  avg_transaction_amount_7d     50000 non-null  float64
 13  f

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [8]:
df = df.drop(['transaction_id', 'user_id'], axis=1)

In [9]:
df['fraud_label'].value_counts()

fraud_label
0    33933
1    16067
Name: count, dtype: int64

# Experiment 1

In [10]:
mlflow.set_experiment("AnomalyDetection")

<Experiment: artifact_location='file:///c:/Users/pc/D/projects/Anomaly-Detection/mlruns/2', creation_time=1769451908856, experiment_id='2', last_update_time=1769451908856, lifecycle_stage='active', name='AnomalyDetection', tags={'mlflow.experimentKind': 'custom_model_development'}>

## Run 1

In [11]:
X = df.drop(['fraud_label'], axis=1)
y = df[['fraud_label']]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=55)
X_test, X_valid, y_test, y_valid = train_test_split(X_temp, y_temp, test_size=0.4, random_state=55)

In [12]:
categories = X.select_dtypes(include=['object']).columns.tolist()
numerical = X.select_dtypes(include=['number']).columns.tolist()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

In [13]:
transformer = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categories),
    ("num", 'passthrough', numerical)
])

In [14]:
params = {
    'C': 1.0,
    'class_weight': None,
    'max_iter': 100,
    'penalty': 'l2'
}

In [15]:
pipe = Pipeline([
    ('preprocess', transformer),
    ('model', LogisticRegression(**params))
])

In [16]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)
y_pred = pipe.predict(X_valid)
report = classification_report(y_pred, y_valid, output_dict=True)

In [17]:
with mlflow.start_run(run_name='LogisticRegression'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_0': report['0']['recall'],
        'recall_1': report['1']['recall'],
        'f1-score': report['macro avg']['f1-score']
    })
   
    mlflow.sklearn.log_model(pipe, name='LogisticRegression')

🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/2/runs/05b060a31fc64fee96475c9db1e6a6f9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


## Run 2

In [18]:
transformer = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categories),
    ("num", 'passthrough', numerical),
    ("std", StandardScaler(), numerical),
    ("pca", PCA(), numerical)
])

In [19]:
pipe = Pipeline([
    ('preprocess', transformer),
    ('model', LogisticRegression(**params))
])

In [20]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)
y_pred = pipe.predict(X_valid)
report = classification_report(y_pred, y_valid, output_dict=True)

In [21]:
with mlflow.start_run(run_name='OHE-LogisticRegression'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_0': report['0']['recall'],
        'recall_1': report['1']['recall'],
        'f1-score': report['macro avg']['f1-score']
    })
   
    mlflow.sklearn.log_model(pipe, name='OHE-LogisticRegression')

🏃 View run OHE-LogisticRegression at: http://127.0.0.1:5000/#/experiments/2/runs/e0060b05870e4c97b932db7fe62eb84d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


## Run 3

In [22]:
from imblearn.pipeline import Pipeline

pipe = Pipeline([
    ('preprocess', transformer),
    ('smote', SMOTE(sampling_strategy='minority')),
    ('model', LogisticRegression(**params))
])

In [23]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)
y_pred = pipe.predict(X_valid)
report = classification_report(y_pred, y_valid, output_dict=True)

In [24]:
with mlflow.start_run(run_name='SMOTE-LogisticRegression'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_0': report['0']['recall'],
        'recall_1': report['1']['recall'],
        'f1-score': report['macro avg']['f1-score']
    })
   
    mlflow.sklearn.log_model(pipe, name='SMOTE-LogisticRegression')

🏃 View run SMOTE-LogisticRegression at: http://127.0.0.1:5000/#/experiments/2/runs/d9ce06e3110e4abea8d5c8a8433715b9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


## Run 4

In [25]:
transformer = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categories),
    ("num", 'passthrough', numerical),
    ("std", StandardScaler(), numerical),
    ("pca", PCA(), numerical)
])

In [26]:
from imblearn.pipeline import Pipeline
pipe = Pipeline([
    ('preprocess', transformer),
    ('adasyn', ADASYN(sampling_strategy='minority')),
    ('model', LogisticRegression(**params))
])

In [27]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)
y_pred = pipe.predict(X_valid)
report = classification_report(y_pred, y_valid, output_dict=True)

In [28]:
with mlflow.start_run(run_name='ADASYN-LogisticRegression'):
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_0': report['0']['recall'],
        'recall_1': report['1']['recall'],
        'f1-score': report['macro avg']['f1-score']
    })
   
    mlflow.sklearn.log_model(pipe, name='ADASYN-LogisticRegression')

🏃 View run ADASYN-LogisticRegression at: http://127.0.0.1:5000/#/experiments/2/runs/4c410c6347a94dcca6b04afcb80988d3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


**After comparing instances, I find that the model from experiment 2 performs better than the others regarding the recall for** `class 1` **, so I'll register it to MLflow model registry to fine-tune it further**

### Registering a model

In [31]:
import mlflow
model_name = input('Model name: ')
run_id = input('Run ID: ')
model_uri = f"runs:/{run_id}/{model_name}"

mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)

Successfully registered model 'OHE-LogisticRegression'.
2026/01/28 11:14:33 WARNING mlflow.tracking._model_registry.fluent: Run with id e0060b05870e4c97b932db7fe62eb84d has no artifacts at artifact path 'OHE-LogisticRegression', registering model based on models:/m-05d95214a34c4db98bb99750541a4352 instead
2026/01/28 11:14:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: OHE-LogisticRegression, version 1
Created version '1' of model 'OHE-LogisticRegression'.


<ModelVersion: aliases=[], creation_timestamp=1769591674145, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1769591674145, metrics=None, model_id=None, name='OHE-LogisticRegression', params=None, run_id='e0060b05870e4c97b932db7fe62eb84d', run_link='', source='models:/m-05d95214a34c4db98bb99750541a4352', status='READY', status_message=None, tags={}, user_id='', version='1'>

# Experiment 2

In [32]:
mlflow.set_experiment('AnomalyDetection-2')

2026/01/28 11:15:12 INFO mlflow.tracking.fluent: Experiment with name 'AnomalyDetection-2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1769591712726, experiment_id='3', last_update_time=1769591712726, lifecycle_stage='active', name='AnomalyDetection-2', tags={}>

In [33]:
transformer = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categories),
    ("num", 'passthrough', numerical),
    ("std", StandardScaler(), numerical),
    ("pca", PCA(), numerical)
])

In [72]:
params = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'max_iter': [100, 500, 700],
    'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
}

In [73]:
# Create a custom scorer to optimize the model on
recall_1 = make_scorer(recall_score, pos_label=1)

In [74]:
search = RandomizedSearchCV(
    LogisticRegression(), 
    param_distributions=params,
    scoring=recall_1,
    cv=5,
    verbose=1,
    random_state=66
)

In [75]:
pipe = Pipeline([
    ('preprocess', transformer),
    ('random-search', search)
])

In [76]:
pipe.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,steps,"[('preprocess', ...), ('random-search', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formattin

In [77]:
y_pred = pipe.predict(X_valid)
report = classification_report(y_pred, y_valid, output_dict=True)
print(classification_report(y_pred, y_valid))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89      5664
           1       0.72      0.79      0.75      2336

    accuracy                           0.85      8000
   macro avg       0.82      0.83      0.82      8000
weighted avg       0.85      0.85      0.85      8000



In [78]:
with mlflow.start_run(run_name='CV-OHE-LogisticRegression'):
    mlflow.log_params(search.best_params_)
    mlflow.log_metrics({
        'accuracy': report['accuracy'],
        'recall_0': report['0']['recall'],
        'recall_1': report['1']['recall'],
        'f1-score': report['macro avg']['f1-score']
    })

    mlflow.sklearn.log_model(
        pipe, 
        name='OHE-LogisticRegression', 
    )

🏃 View run CV-OHE-LogisticRegression at: http://127.0.0.1:5000/#/experiments/3/runs/344f3876288f4c1e8ef227ed25355d34
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


In [79]:
# Update the existing registered model
run_id = input('Run ID: ')
model_name = input('Model name: ')

model_uri = f'runs:/{run_id}/{model_name}'

mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'OHE-LogisticRegression' already exists. Creating a new version of this model...
2026/01/28 15:51:15 WARNING mlflow.tracking._model_registry.fluent: Run with id 31e1c31c6a7841bcaff24c2722a4066a has no artifacts at artifact path 'OHE-LogisticRegression', registering model based on models:/m-d5d6f95ee67a4a17b049b858bc4a4b03 instead
2026/01/28 15:51:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: OHE-LogisticRegression, version 4
Created version '4' of model 'OHE-LogisticRegression'.


<ModelVersion: aliases=[], creation_timestamp=1769608276185, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1769608276185, metrics=None, model_id=None, name='OHE-LogisticRegression', params=None, run_id='31e1c31c6a7841bcaff24c2722a4066a', run_link='', source='models:/m-d5d6f95ee67a4a17b049b858bc4a4b03', status='READY', status_message=None, tags={}, user_id='', version='4'>

### Loading a specific version of a registered model

In [80]:
run_id = input('Run ID: ')
model_name = input('Model name: ')

model = mlflow.sklearn.load_model(f'runs:/{run_id}/{model_name}')

In [81]:
model

,steps,"[('preprocess', ...), ('random-search', ...)]"
,transform_input,None
,memory,None
,verbose,False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formattin

### Testing the best Logistic Regression model on the test set

In [82]:

model.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_pred, y_test))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      8538
           1       0.71      0.80      0.75      3462

    accuracy                           0.85     12000
   macro avg       0.81      0.83      0.82     12000
weighted avg       0.85      0.85      0.85     12000

